In [1]:
import cv2
import numpy as np
import os


In [3]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1,  minNeighbors=5, minSize=(30, 30))


In [ ]:
base_path = 'C:/Users/jungh/final_project/seasons_img'


In [ ]:
for root, dirs, files in os.walk(base_path): # os.walk()를 사용하면 모든 하위 폴더와 파일을 탐색 할 수 있다  하위의 폴더들을 for문으로 탐색할 수 있게 해줌

    for file in files:
        if file.lower().endswith('.jpg'):
            image_path = os.path.join(root, file)

            # 이미지 변환 코드
            image = cv2.imread(image_path)
            # 이미지 HSV 색 공간 변환
            img_HSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17, 170, 255))
            HSV_mask = cv2.morphologyEx(HSV_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

            # 이미지 YCrCb 색 공간 변환
            img_YCrCb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
            YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255, 180, 135))
            YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

            # 얼굴과 머리카락 영역의 마스크 생성
            global_mask = cv2.bitwise_and(YCrCb_mask, HSV_mask)
            global_mask = cv2.medianBlur(global_mask, 3)
            global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, np.ones((4, 4), np.uint8))

            # 머리카락 영역의 마스크 생성
            spatial_radius = 10
            color_radius = 10
            max_pyramid_level = 1
            segmented = cv2.pyrMeanShiftFiltering(image, spatial_radius, color_radius, maxLevel=max_pyramid_level)
            segmented_gray = cv2.cvtColor(segmented, cv2.COLOR_BGR2GRAY)
            _, hair_mask = cv2.threshold(segmented_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
            kernel = np.ones((5, 5), np.uint8)
            hair_mask = cv2.morphologyEx(hair_mask, cv2.MORPH_OPEN, kernel)

            # 얼굴과 머리카락 영역을 합쳐서 최종 마스크 생성
            final_mask = cv2.bitwise_or(global_mask, hair_mask)


            # 최종 마스크를 사용하여 얼굴과 머리카락만 남기고 나머지는 흰색으로 처리
            result = cv2.bitwise_and(image, image, mask=final_mask)
            
            # 이미지 저장 코드
            output_dir = os.path.join(root, 'processed')
            os.makedirs(output_dir, exist_ok=True)
            output_path = os.path.join(output_dir, file)
            cv2.imwrite(output_path, result)